In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.llms import HuggingFaceHub
from crewai import LLM
import os

# Setting up the LLMs to use for our agents.
# We can also keep different LLMs for different agents

# llm = HuggingFaceHub(
#     repo_id = "mistralai/Mistral-7B-Instruct-v0.3",
#     huggingfacehub_api_token = os.getenv("HF_TOKEN"),
#     task = "text-generation",
# )

llm = LLM(model = "openai/gpt-4o-mini")

In [5]:
from crewai import Agent, Task, Crew

In [6]:
support_agent = Agent(
    role = "Senior Support Representative",
	goal = "Be the most friendly and helpful support representative in your team",
	backstory = """You work at crewAI (https://crewai.com) and are now working on providing support to {customer}, a super important customer
        for your company. You need to make sure that you provide the best support! 
        Make sure to provide full complete answers, and make no assumptions.""",
	allow_delegation = False,
	verbose = True
)

In [ ]:
support_quality_assurance_agent = Agent(
	role = "Support Quality Assurance Specialist",
	goal = "Get recognition for providing the best support quality assurance in your team",
	backstory = """You work at crewAI (https://crewai.com) and are now working with your team on a request from {customer} ensuring that the support representative 
        is providing the best support possible.
        You need to make sure that the support representative is providing full complete answers, and make no assumptions.""",
	verbose = True
)

In [8]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

d:\Anaconda\envs\crewai_env\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
d:\Anaconda\envs\crewai_env\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
d:\Anaconda\envs\crewai_env\Lib\site-packages\crewai_tools\tools\selenium_scraping_tool\selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 

In [9]:
docs_scrape_tool = ScrapeWebsiteTool(website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/")

In [10]:
inquiry_resolution = Task(
    description = 
    """{customer} just reached out with a super important ask:
	{inquiry}
    {person} from {customer} is the one that reached out.
	Make sure to use everything you know to provide the best support possible. 
    You must strive to provide a complete and accurate response to the customer's inquiry.""",
    expected_output = """A detailed, informative response to the customer's inquiry that addresses all aspects of their question.
    The response should include references to everything you used to find the answer, including external data or solutions. 
    Ensure the answer is complete, leaving no questions unanswered, and maintain a helpful and friendly tone throughout.""",
	tools = [docs_scrape_tool],
    agent = support_agent,
)

In [ ]:
quality_assurance_review = Task(
    description = """Review the response drafted by the Senior Support Representative for {customer}'s inquiry.
        Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
        Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
        Check for references and sources used to find the information, ensuring the response is well-supported and leaves no questions unanswered.""",
    expected_output =
        """A final, detailed, and informative response ready to be sent to the customer.
        This response should fully address the customer's inquiry, incorporating all relevant feedback and improvements.
		Don't be too formal, we are a chill and cool company but maintain a professional and friendly tone throughout.""",
    agent = support_quality_assurance_agent,
)


In [ ]:
crew = Crew(
  agents = [support_agent, support_quality_assurance_agent],
  tasks = [inquiry_resolution, quality_assurance_review],
  verbose = True,
  memory = True
)

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs = inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)